<a href="https://colab.research.google.com/github/ManJ-PC/py_data_science/blob/master/bdcc2324_pycuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming GPUs in Python3 using PyCuda
**[Big Data and Cloud Computing]**

Most machines have available GPUs which we should take advantage of. GPUs can greatly accelerate execution if well programmed. Here we go through some examples.

The example given in this notebook will only take advantage of a GPU if you have cuda installed and if you have a graphics card that is cuda-enabled. Please check [this wikipedia site](https://en.wikipedia.org/wiki/CUDA) or the official [nvidia site] (https://developer.nvidia.com/cuda-gpus) for more detail about compatibility. If you run this notebool in colab, it already provides you with a GPU in its runtime environment.

__References__:

- [Tutorial on PyCuda](https://documen.tician.de/pycuda/tutorial.html)
- [CUDA programming guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)

In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661206 sha256=7ba63b6cec1c4ba23e79eb84584de84f9e72af0f325711f9f967ad0cf705078a
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


Let's start with a very simple code. We will create two vectors (A and B) and add them producing a third vector (C). The kernel code is reproduced below: (not executable yet)

In [2]:
// Kernel definition
__global__ void VecAdd(float* A, float* B, float* C)
{
    int i = threadIdx.x;
    C[i] = A[i] + B[i];
}


SyntaxError: invalid syntax (<ipython-input-2-ebfc4123d45d>, line 1)

Let's make it executable by integrating it in a Python code. The array is created using `numpy`. We conveniently use numpy when working with GPUs, because GPUs can work very well on number crunching operations.

(**NOTE**: if running in colab, you may need to configure your runtime to have a GPU. Click on the "Runtime" menu and then "Change Runtime type".)

In [ ]:
############################
# GPU version
############################

# need to import modules
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
# end
import numpy
from time import time

# begin timing
start_time = time()
########
N = 16
numpy.random.seed(123)
a = numpy.random.rand(N)
a = a.astype(numpy.float32)
b = numpy.random.rand(N)
b = b.astype(numpy.float32)
c = numpy.zeros(shape=(1,N),dtype=numpy.float32)

# need to allocate memory in the GPU to fit our array
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)
# a_tid = cuda.mem_alloc(b.nbytes)
# a_bid = cuda.mem_alloc(c.nbytes)
# need to copy our array to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)
# cuda.memcpy_htod(a_tid, b)
# cuda.memcpy_htod(a_bid, c)

# here it is the kernel that will run the addition in the GPU
mod = SourceModule("""
    __global__ void VecAdd(float* A, float* B, float* C)
    {
        int i = threadIdx.x;
        C[i] = A[i] + B[i];
    }
 """)
func = mod.get_function("VecAdd")
# now, we define the shape
func(a_gpu, b_gpu, c_gpu, block=(N,1,1))

# create space in the host memory to receive results
#c = numpy.empty_like(c)
#c_tid_result = numpy.empty_like(c)

# copy results from the GPU memory to the host memory
cuda.memcpy_dtoh(c, c_gpu)

# end timing
print(round(time() - start_time,8), 'seconds')
########

# print("tid  bid\n")
# for i in range(len(b)):
#   s = ""
#   for j in range(len(b[i])):
#      s += str(b[i,j])+"  "+str(c[i,j])+"  "
#   print(s)
print(c)
print(a)
print(b)
(a+b==c)


Now, let's play with a bidimensional array with dimensions RxC. In this example, we create the matrix using `numpy` again. The function simply multiplies each cell of the matrix by 2. We create first a sequential version and next a cuda version.

In [ ]:
############################
# Sequential version
############################

import numpy
from time import time

R = 16
C = 16
# begin timing
start_time = time()
########

numpy.random.seed(123)
a = numpy.random.randn(R,C)
a_doubled = a*2

# end timing
print(round(time() - start_time,8), 'seconds')
########

print(a_doubled)
print(a)
a*2==a_doubled

Now, let's "decorate" this code to use a GPU to compute all multiplications in parallel. In this example, we use auxiliary arrays that will contain the thread IDs and block IDs, so that we can inspect the number of threads working and what each one is doing. Thread IDs Block IDs are stored in the GPU, in vectors called a_tid and a_bid, respectively. In the host they are called b and c. Our matrix is called `a` in the host and `a_gpu` in the GPU.

In [ ]:
############################
# GPU version
############################

# need to import modules
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
# end
import numpy
from time import time

# begin timing
start_time = time()
########
R = 16
C = 16
numpy.random.seed(123)
a = numpy.random.randn(R,C)
a = a.astype(numpy.float32)
b = numpy.zeros(shape=(R,C),dtype=numpy.uint32)
c = numpy.zeros(shape=(R,C),dtype=numpy.uint32)

# need to allocate memory in the GPU to fit our array
a_gpu = cuda.mem_alloc(a.nbytes)
a_tid = cuda.mem_alloc(b.nbytes)
a_bid = cuda.mem_alloc(c.nbytes)
# need to copy our array to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(a_tid, b)
cuda.memcpy_htod(a_bid, c)

# here it is the kernel that will run the multiplication in the GPU
mod = SourceModule("""
  __global__ void doublify(float *a, uint *a_tid, uint *a_bid)
  {
    int idx = threadIdx.x + threadIdx.y*16;
    a[idx] *= 2;
    a_tid[idx] = threadIdx.x + blockDim.x * threadIdx.y;
    a_bid[idx] = blockDim.x * blockDim.y;

  }
  """)
func = mod.get_function("doublify")
# now, we define the shape
func(a_gpu, a_tid, a_bid, block=(1,R,C))

# create space in the host memory to receive results
a_doubled = numpy.empty_like(a)
a_tid_result = numpy.empty_like(b)

# copy results from the GPU memory to the host memory
cuda.memcpy_dtoh(a_doubled, a_gpu)
cuda.memcpy_dtoh(b, a_tid)
cuda.memcpy_dtoh(c, a_bid)


# end timing
print(round(time() - start_time,8), 'seconds')
########

print("tid  bid\n")
for i in range(len(b)):
  s = ""
  for j in range(len(b[i])):
     s += str(b[i,j])+"  "+str(c[i,j])+"  "
  print(s)
print(a_doubled)
print(a)
(a*2==a_doubled)



# Q1: Have you noticed that the array dimension, the grid shape and the offset calculated by each thread are all related? Increase the dimension of the arrays in both programs and play with the offset and with the `dim3`. Report what you understood about the distribution of threads and blocks and how the threads execute the kernel operations for each program.

Knowing more about the device and the system...

In [ ]:
!nvidia-smi

Collecting device properties using `pycuda`:

In [ ]:
import pycuda.driver as drv
import pycuda.autoinit
print("PyCUDA version:", pycuda.VERSION_TEXT)
print("Device(s) found:", drv.Device.count())
for ordinal in range(drv.Device.count()):
  dev = drv.Device(ordinal)
  print("Device number:", ordinal, "Device name:", dev.name())
  print("Compute capability: ", dev.compute_capability())
  print("Max threads per block:", dev.max_threads_per_block)
  print("Max block_dim_x", dev.MAX_BLOCK_DIM_X)
  print("Max block_dim_y", dev.MAX_BLOCK_DIM_Y)
  print("Max block_dim_z", dev.MAX_BLOCK_DIM_Z)
  print("Max grid_dim_x",dev.MAX_GRID_DIM_X)
  print("Max grid_dim_y",dev.MAX_GRID_DIM_Y)
  print("Max grid_dim_z",dev.MAX_GRID_DIM_Z)
  print("Max total constant memory",dev.TOTAL_CONSTANT_MEMORY)
  print("Max warp size",dev.WARP_SIZE)
  print(dev.get_attributes())


# Q2: inspect the nvidia-smi command and check how you can obtain the same information or more detailed information using the command line.